In [ ]:
from dotenv import load_dotenv
from pathlib import Path
from openai import AsyncAzureOpenAI
import os

In [79]:
ENV_PATH = Path().absolute().parent / ".env" / "dev.env"

In [80]:
load_dotenv(ENV_PATH, override=True)

True

In [81]:
DEPLOYMENT_NAME = "gpt-5-mini"

In [82]:
CONTEXT = [
    {
        "role": "system",
        "content": "You are a helpful assistant.",
    },
    {
        "role": "user",
        "content": "I am going to Prague, what should I see tell me top 3 things?",
    },
]

## Method 1: API Key Authentication

The simplest authentication method using an API key stored in environment variables. Quick to set up but less secure for production environments as keys can be leaked or compromised.

In [ ]:
CLIENT = AsyncAzureOpenAI(
  azure_endpoint = os.environ["OPENAI_API_BASE_V1"], 
  api_key=os.environ["OPENAI_API_BASE_V1_KEY"],
  api_version=os.environ["OPENAI_API_VERSION"],
)


In [84]:
async def completion_create(client: AsyncAzureOpenAI, messages: list[dict]) -> str:
    response = await client.chat.completions.create(
        messages=messages,
        max_completion_tokens=2024,
        model=DEPLOYMENT_NAME,
        reasoning_effort="minimal",
        verbosity="low",
        seed=42,
    )
    return response


In [85]:
response = await completion_create(client=CLIENT, messages=CONTEXT)

In [86]:
print(response.choices[0].message.content)

Great choice — Prague is beautiful! Here are the top 3 must-see sights, with a brief reason to visit each and a tip for making the most of it:

1. Prague Castle (Pražský hrad)
   - Why: Iconic medieval fortress complex with St. Vitus Cathedral, royal palaces, historic streets, and panoramic views over the city.
   - Tip: Arrive early to avoid crowds and take the route down through Golden Lane. Buy a combined ticket (several circuits available) if you want interior access.

2. Charles Bridge (Karlův most)
   - Why: Famous 14th-century stone bridge lined with baroque statues, offering stunning views of the Vltava River, Prague Castle, and Old Town.
   - Tip: Visit at sunrise or late evening for fewer people and beautiful light; watch street musicians and local artists but be mindful of pickpockets in crowded times.

3. Old Town Square & Astronomical Clock (Staroměstské náměstí)
   - Why: The historic heart of Prague with colorful Gothic and Baroque buildings, lively cafés, the Church of 

## Method 2: Azure AD Token Provider (DefaultAzureCredential)

Uses `DefaultAzureCredential` which automatically tries multiple authentication methods (environment variables, managed identity, Azure CLI, etc.). Recommended for production as it doesn't require storing secrets and supports seamless identity management.

In [48]:
# Alternative: Using Azure AD token provider (recommended for production)
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [49]:
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

client = AsyncAzureOpenAI(
    azure_endpoint=os.environ["OPENAI_API_BASE_V1"],
    azure_ad_token_provider=token_provider,
    api_version=os.environ["OPENAI_API_VERSION"],
)

In [ ]:
response = await completion_create(client=CLIENT, messages=CONTEXT)

In [ ]:
print(response.choices[0].message.content)

## Method 3 (A): Certificate-Based Authentication

Uses X.509 certificates for authentication via `CertificateCredential`. Ideal for automated systems, service principals, and scenarios where you need non-interactive authentication with strong security guarantees.

In [66]:
# Alternative: Using custom certificate-based token provider
from typing import Callable
from azure.identity import CertificateCredential

In [ ]:
def get_cert_token_provider(
    tenant_id: str,
    client_id: str,
    certificate_data: bytes,
    scope: str = "https://cognitiveservices.azure.com/.default",
) -> Callable[[], str]:
    """Returns a callable that provides a bearer token using certificate-based authentication."""
    credential = CertificateCredential(
        tenant_id=tenant_id,
        client_id=client_id,
        certificate_data=certificate_data,
    )

    def token_provider() -> str:
        token = credential.get_token(scope)
        return token.token

    return token_provider

In [88]:
# Load certificate data from environment variables (private + public key)
certificate_data = os.environ["PRIVATE_CERT_KEY"].encode() + b"\n" + os.environ["PUBLIC_CERT_KEY"].encode()

cert_token_provider = get_cert_token_provider(
    tenant_id=os.environ["TENANT_ID"],
    client_id=os.environ["CLIENT_ID"],
    certificate_data=certificate_data,
)

client = AsyncAzureOpenAI(
    azure_endpoint=os.environ["OPENAI_API_BASE_V1"],
    azure_ad_token_provider=cert_token_provider,
    api_version=os.environ["OPENAI_API_VERSION"],
)

In [89]:
response = await completion_create(client=CLIENT, messages=CONTEXT)

In [90]:
print(response.choices[0].message.content)

Great choice — Prague is beautiful! Here are the top 3 must-see sights, with a brief reason to visit each and a tip:

1. Prague Castle (Pražský hrad)
   - Why: One of the largest castle complexes in the world, it includes St. Vitus Cathedral, the Old Royal Palace, and Golden Lane. It gives a sweeping view over the city.
   - Tip: Arrive early to avoid crowds and buy a timed ticket for the castle circuit (or explore the grounds for free).

2. Charles Bridge (Karlův most)
   - Why: Iconic 14th-century pedestrian bridge lined with baroque statues, connecting Old Town and Prague Castle. Great for photos, street musicians, and historic atmosphere.
   - Tip: Visit at sunrise or late evening to avoid the busiest times; beware of pickpockets in crowded stretches.

3. Old Town Square & Astronomical Clock (Staroměstské náměstí)
   - Why: Prague’s historic heart with Gothic and Baroque architecture, lively cafés, and the medieval Astronomical Clock that performs hourly.
   - Tip: Watch the clock 

## Method 3B: Persisted AD Token Authentication

Retrieves an Azure AD token once and reuses it for multiple requests as long as it remains valid. This reduces authentication overhead by avoiding repeated token generation, useful for batch operations or high-frequency API calls within the token's lifetime.

In [91]:
ad_token = cert_token_provider()

In [92]:
client = AsyncAzureOpenAI(
    azure_endpoint=os.environ["OPENAI_API_BASE_V1"],
    azure_ad_token=ad_token,
    api_version=os.environ["OPENAI_API_VERSION"],
)

In [93]:
response = await completion_create(client=CLIENT, messages=CONTEXT)

In [94]:
print(response.choices[0].message.content)

Great choice — Prague is beautiful! If I had to pick the top three must-see sights, they are:

1. Prague Castle and St. Vitus Cathedral
   - The castle complex (one of the largest in the world) sits on a hill overlooking the city. Explore the courtyards, the Old Royal Palace, Golden Lane, and don’t miss St. Vitus Cathedral — its stained glass, Gothic architecture, and views from the towers are highlights.

2. Charles Bridge
   - This historic 14th-century stone bridge links Old Town and Lesser Town. Walk across early morning or at dusk to avoid crowds, admire the baroque statues lining the bridge, and enjoy street musicians and views of the Vltava River and Prague Castle.

3. Old Town Square and the Astronomical Clock
   - The heart of Prague’s Old Town: colorful Gothic and Baroque buildings, the Týn Church, and the medieval Astronomical Clock (Orloj). On the hour the clock’s mechanical procession draws a crowd. The square is also surrounded by cafes where you can relax and people-watc